<a href="https://colab.research.google.com/github/mamadoz79/Neural_networks_CA3/blob/main/CA3_part2_NN_Temperature_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import pandas as pd
import os
import itertools
from sklearn.model_selection import train_test_split
from scipy import *
from scipy.linalg import norm, pinv
from scipy.optimize import minimize
from sklearn.cluster import MiniBatchKMeans as kmeans
from matplotlib import pyplot as plt
from numpy import random
import timeit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


data

In [ ]:
def normalize(df_min_max_scaled):
    for column in df_min_max_scaled.columns:
        df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())
    return df_min_max_scaled

In [ ]:
data_sets = {}
for i in os.listdir('/content/drive/MyDrive/Time-series/'):
    name = i.split('.')[0]
    data_sets.update({name : pd.read_excel(f"/content/drive/MyDrive/Time-series/{i}", header=None)})
    data_sets[name] = {"target": np.array(normalize(data_sets[name].iloc[:,-1:])), "data": np.array(normalize(data_sets[name].iloc[:,:-1]))}

<ipython-input-3-d8db9858f8fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())
<ipython-input-3-d8db9858f8fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_sets['Temperature Dataset']['data'], data_sets['Temperature Dataset']['target'], test_size=0.3)

GMDH

In [ ]:
class GMDH:
    def __init__(self):
        self.weights = np.array([random.random() for i in range(6)])
        self.etha = 0.01
        self.outputs = []
        self.test_output = []
        self.error_test = []
        self.error_train = []
        self.local_error_test = []
        self.local_error_train = []

    def feed_forward(self, X, target, test_flag):
        self.x1, self.x2 = X
        self.target = target
        self.output = self.dot_product()
        self.error = self.calc_error()
        if not test_flag:
            self.outputs.append(self.output)
            self.local_error_train.append(self.error)
        else:
            self.test_output.append(self.output)
            self.local_error_test.append(self.error)

    def make_error_list(self, arg):
        if arg == 'test':
            self.error_test.append(self.local_error_test)
            self.local_error_test = []
        elif arg == 'train':
            self.error_train.append(self.local_error_train)
            self.local_error_train = []

    def back_propag(self):
        self.weights -= (np.array([1, self.x1, self.x2, self.x1**2, self.x1*self.x2, self.x2**2]) * self.error * -1 * self.etha)

    def dot_product(self):
        return self.weights[0] + self.weights[1]*self.x1 + self.weights[2]*self.x2 + self.weights[3]*(self.x1**2) + self.weights[4]*(self.x1*self.x2) + self.weights[5]*(self.x2**2)

    def calc_error(self):
        return self.target - self.output


In [ ]:
GMDH_neurons_layer_1 = [GMDH() for i in range(3)]

In [ ]:
def GMDH_training(Data, Target, Data_test, Target_test, epoch=200, GMDH_neurons=GMDH_neurons_layer_1):
    for ep in range(epoch):
        for neuron in GMDH_neurons:
            neuron.outputs = []
        #train part
        for (data, target) in zip(Data, Target):
            [i.feed_forward(j, target[0], False) for (i, j) in zip(GMDH_neurons, list(itertools.combinations(data, 2)))]
            [i.back_propag() for i in GMDH_neurons]
        for ne in GMDH_neurons:
            ne.make_error_list('train')

        for neuron in GMDH_neurons:
            neuron.test_output = []

        for (data, target) in zip(Data_test, Target_test):
            [i.feed_forward(j, target[0], True) for (i, j) in zip(GMDH_neurons, list(itertools.combinations(data, 2)))]
        for ne in GMDH_neurons:
            ne.make_error_list('test')

    for neuron in GMDH_neurons:
        neuron.outputs = np.array(neuron.outputs)

In [ ]:
GMDH_training(X_train, y_train, X_test, y_test)

In [ ]:
GMDH_neurons_layer_2 = [GMDH() for i in range(3)]

In [ ]:
GMDH_neurons_layer_2_x_train = np.array(list(zip(*[i.outputs for i in GMDH_neurons_layer_1])))

In [ ]:
GMDH_neurons_layer_2_x_test = np.array(list(zip(*[i.test_output for i in GMDH_neurons_layer_1])))

In [ ]:
GMDH_training(GMDH_neurons_layer_2_x_train, y_train, GMDH_neurons_layer_2_x_test, y_test, GMDH_neurons=GMDH_neurons_layer_2)

In [176]:
mlp_x_train = np.array(list(zip(*[i.outputs for i in GMDH_neurons_layer_2])))
mlp_x_test = np.array(list(zip(*[i.test_output for i in GMDH_neurons_layer_2])))

MLP

In [400]:
class MLP:
    def __init__(self):
        self.weights = np.array([random.random() for i in range(3)])
        self.etha = 0.01
        self.outputs = []
        self.test_output = []
        self.error_test = []
        self.error_train = []
        self.local_error_test = []
        self.local_error_train = []

    def feed_forward(self, X, test_flag):
        self.x = X
        self.net = sum(self.weight * self.x)
        self.output = self.sigmoid(self.net)
        if not test_flag:
            self.outputs.append(self.output)
            self.local_error_train.append(self.error)
        else:
            self.test_output.append(self.output)
            self.local_error_test.append(self.error)

    def make_error_list(self, arg):
        if arg == 'test':
            self.error_test.append(self.local_error_test)
            self.local_error_test = []
        elif arg == 'train':
            self.error_train.append(self.local_error_train)
            self.local_error_train = []

    def back_propag(self, rgh_flag):
        pass

    def sigmoid(self, z, first_derivative=False):
        if first_derivative:
            return z*(1.0-z)
        return 1.0/(1.0+np.exp(-z))


In [ ]:
def MLP_training(Data, Target, Data_test, Target_test, epoch=200, mlp_neurons=mlp_neurons):
    for ep in range(epoch):
        for neuron in mlp_neurons:
            neuron.outputs = []
        #train part
        for data in Data:
            [i.feed_forward(data, False) for i in mlp_neurons]
            #[i.back_propag() for i in mlp_neurons]
        for ne in mlp_neurons:
            ne.make_error_list('train')

        for neuron in mlp_neurons:
            neuron.test_output = []
        '''
        for (data, target) in zip(Data_test, Target_test):
            [i.feed_forward(j, target[0], True) for (i, j) in zip(mlp_neurons, list(itertools.combinations(data, 2)))]
        for ne in mlp_neurons:
            ne.make_error_list('test')
        '''
    for neuron in mlp_neurons:
        neuron.outputs = np.array(neuron.outputs)

In [404]:
class Rough:
    def __init__(self, inp_size):
        self.weightsL = np.array([random.random() for i in range(inp_size)])
        self.weightsU = np.array([random.random() for i in range(inp_size)])
        self.alpha = random.rand()
        self.beta = random.rand()
        self.etha = 0.01
        self.outputsL = []
        self.outputsU = []

        self.test_output = []
        self.error_test = []
        self.error_train = []
        self.local_error_test = []
        self.local_error_train = []

    def feed_forward(self, X, target, test_flag):
        self.x = X
        self.target = target
        self.netU = sum(self.weightU * self.x)
        self.netL = sum(self.weightL * self.x)
        self.outputU = max(self.netU, self.netL)
        self.outputL = min(self.netU, self.netL)
        self.flag = self.netU > self.netL
        self.output = self.alpha * self.outputU + self.beta * self.outputL
        self.error = self.calc_error()
        if not test_flag:
            self.outputs.append(self.output)
            self.local_error_train.append(self.error)

        else:
            self.test_output.append(self.output)
            self.local_error_test.append(self.error)

    def make_error_list(self, arg):
        if arg == 'test':
            self.error_test.append(self.local_error_test)
            self.local_error_test = []
        elif arg == 'train':
            self.error_train.append(self.local_error_train)
            self.local_error_train = []

    def back_propag(self, rgh_flag):
        self.alpha += self.etha * self.error * self.outputU
        self.beta += self.etha * self.error * self.outputL
        self.weightsU += self.etha * self.error * self.x * (self.flag * self.alpha + (1 - self.flag) * self.beta)
        self.weightsL += self.etha * self.error * self.x * ((1 - self.flag) * self.alpha + self.flag * self.beta)


    def linear(self, x, derivative=False):
        if derivative:
            return 1
        return x

    def calc_error(self):
        return self.target - self.output

In [ ]:
def Rough_training(Data, Target, Data_test, Target_test, rough_neuron=rough_neuron):
    for neuron in rough_neuron:
        neuron.outputs = []
    #train part
    for data in Data:
        [i.feed_forward(data, False) for i in rough_neuron]
        #[i.back_propag() for i in rough_neuron]
    for ne in rough_neuron:
        ne.make_error_list('train')

    for neuron in rough_neuron:
        neuron.test_output = []
    '''
    for (data, target) in zip(Data_test, Target_test):
        [i.feed_forward(j, target[0], True) for (i, j) in zip(rough_neuron, list(itertools.combinations(data, 2)))]
    for ne in rough_neuron:
        ne.make_error_list('test')
    '''
    for neuron in rough_neuron:
        neuron.outputs = np.array(neuron.outputs)

In [405]:
mlp_neurons = [MLP() for i in range(5)]

In [406]:
rough_neuron = Rough(inp_size=5)

In [408]:
for i in mlp_x_train:
    mlp.feed_forward(i)

[]